# Source Code

In [104]:
import numpy as np


def powerMethod(A, initial=None, numberOfIterations=100, minEigen=False):
    """
    A function to calculate largest eigen value and eigen vector of a matrix

    Args: 
            A (numpy array): Matrix to get eigen value/vector (size n*n)
            initial (numpy array): Vector used as initial guess of eigen vector (size 1*n)
            numberOfIterations (int): Number of iterations untill stopping
            minEigen (boolean): Calculate smallest eigen value or largest

    Returns:
            eigenValue (float)
            eigenVector (numpy array)

    Raises:
            TypeError
            ValueError

    """

    ##Validate input##

    # negative number of iterations
    if numberOfIterations <= 0:
        raise ValueError("Number of iterations should be positive")

    # invalid type of matrix A
    if not isinstance(A, np.ndarray):
        raise TypeError("A should be of type numpy.ndarray")

    # invalid type of vector initial
    if (initial is not None) and (not isinstance(A, np.ndarray)):
        raise TypeError("initial should be of type numpy.ndarray")

    # check initial is 1d array
    if (initial is not None) and (initial.ndim != 1):
        raise TypeError("initial should be 1D array")

    # number of iterations is not integer
    if not isinstance(numberOfIterations, int):
        raise TypeError("Number of iterations should be integer")

    #matrix is not square
    if A.shape[0] != A.shape[1]:
        raise TypeError("Matrix should be square")
        
    # check size of initial vector
    if (initial is not None) and (initial.shape[0] != A.shape[1]):
        raise TypeError("Length of initial vector should match size of matrix")
    
    
    ##Calculate eigen pair##
    
    # dimension of matrix
    n = A.shape[0]
    
    if minEigen:
        try:
            A = np.linalg.inv(A)
        except:
            #Matrix is not invertible
            raise TypeError("Matrix should be invertible")

    if initial is None:
        # initial guess for eigen vector
        x = np.ones(n, dtype=np.float32)
    else:
        x = initial.copy()
        x = x.astype(np.float32)

    # iterate to get eigen value/eigen vector
    for i in range(numberOfIterations):
        x = np.dot(A, x)
        eigenValue = x[np.argmax(np.abs(x))]
        x /= eigenValue

    if(minEigen and eigenValue != 0):
        eigenValue = 1/eigenValue

    # Eigen Value, Eigen Vector
    return eigenValue, x

In [105]:
def deflate(A):
    """
    A function to calculate largest eigen value and eigen vector of a matrix

    Args: 
            A (numpy array): Matrix to get deflated matrix

    Returns:
            deflatedMat (numpy array): Deflated matrix

    Raises:
            TypeError
            ValueError

    """
    eigenValue, eigenVector = powerMethod(A)
    norm = np.sum(np.square(eigenVector))
    return A-eigenValue/norm*np.outer(eigenVector, eigenVector.T)

# Test Cases

In [106]:
np.set_printoptions(suppress=True)

mat = np.array([[2, -1, 0, 0],
                [-1, 4, -1, 0],
                [0, -1, 4, -1],
                [0, 0, -1, 2]])

# Example of initial guess
init = np.array([2, -1, 0, 0])
e, x = powerMethod(mat, initial=init)
print(e)
print(x)

5.302775637731995
[-0.30277564  1.         -1.          0.30277564]


In [107]:
mat = np.array([[1, 3],
                [2, 2]])
e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

4.0
[1. 1.]
[[-1.  1.]
 [ 0.  0.]]


In [108]:
mat = np.array([[-7, 2],
                [8, -1]])

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

mat = deflate(mat)

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

-9.0
[ 1. -1.]
[[-2.5 -2.5]
 [ 3.5  3.5]]
1.0000000000000009
[-0.71428571  1.        ]
[[-2.83783784 -2.02702703]
 [ 3.97297297  2.83783784]]


In [109]:
mat = np.array([[3, 2, 0],
                [2, 4, 2],
                [0, 2, 5]])

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

e, x = powerMethod(deflate(mat), numberOfIterations=100)
print(e)
print(x)

e, x = powerMethod(mat, minEigen=True)
print(e)
print(x)

7.0
[0.5 1.  1. ]
[[ 2.22222222  0.44444444 -1.55555556]
 [ 0.44444444  0.88888889 -1.11111111]
 [-1.55555556 -1.11111111  1.88888889]]
4.0
[-1.  -0.5  1. ]
1.0000000000000002
[ 1.  -1.   0.5]


In [110]:
mat = np.array([[3, 0],
                [0, 2]])

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

e, x = powerMethod(deflate(mat))
print(e)
print(x)

3.0
[1. 0.]
[[ 0. -0.]
 [-0.  2.]]
2.0
[-0.  1.]


In [111]:
mat = np.array([[-4, 14, 0],
                [-5, 13, 0],
                [-1, 0, 2]])

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

e, x = powerMethod(deflate(mat))
print(e)
print(x)

e, x = powerMethod(mat, minEigen=True)
print(e)
print(x)

6.000000000000002
[ 1.          0.71428571 -0.25      ]
[[-7.81508516 11.27493917  0.95377129]
 [-7.72506083 11.05352798  0.68126521]
 [-0.04622871  0.68126521  1.76155718]]
2.9999999999999902
[1.         0.91958042 0.46853147]
2.0
[-0. -0.  1.]


In [112]:
mat = np.array([[1, 1],
                [0, -1]])

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

0.5
[1. 1.]
[[ 0.75  0.75]
 [-0.25 -1.25]]


In [113]:
mat = np.array([[87, 270, -12, -49, -276, 40],
                [-14, -45, 6, 10, 46, -4],
                [-50, -156, 4, 25, 162, -25],
                [94, 294, -5, -47, -306, 49],
                [1, 1, 3, 1, 0, 2],
                [16, 48, 1, -6, -48, 8]])

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(mat))

3.9999999999999076
[ 0.33333333  0.         -0.66666667  1.          0.          0.33333333]
[[  86.73333333  270.          -11.46666667  -49.8        -276.
    39.73333333]
 [ -14.          -45.            6.           10.           46.
    -4.        ]
 [ -49.46666667 -156.            2.93333333   26.6         162.
   -24.46666667]
 [  93.2         294.           -3.4         -49.4        -306.
    48.2       ]
 [   1.            1.            3.            1.           -0.
     2.        ]
 [  15.73333333   48.            1.53333333   -6.8         -48.
     7.73333333]]


In [114]:
mat = np.array([[-7, 0, 0],
                [0, -1, 0],
                [0, 0, -25]])
print(mat.T)
e, x = powerMethod(mat)
print(e)
print(x)

deflat = deflate(mat)
print(deflat)

mat = deflat

e, x = powerMethod(mat)
print(e)
print(x)

deflat = deflate(mat)
print(deflat)

mat = deflat

e, x = powerMethod(mat)
print(e)
print(x)

print(deflate(deflat))

[[ -7   0   0]
 [  0  -1   0]
 [  0   0 -25]]
-25.0
[0. 0. 1.]
[[-7.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
-7.0
[ 1.  0. -0.]
[[ 0.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  0.]]
-1.0
[-0.  1. -0.]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [115]:
mat = np.array([[1, 1, -1],
                [1, 2, 1],
                [2, -1, 1]])
e, x = powerMethod(mat)
print(e)
print(x)

2.639802004232656
[0.56298354 1.         0.07681847]


In [116]:
mat = np.array([[-2, 1, 1],
                [3, -2, 0],
                [1, 3, 1]])
e, x = powerMethod(mat)
print(e)
print(x)

-3.0
[-0.33333333  1.         -0.66666667]


In [117]:
mat = np.array([[3, -5],
                [-2, 4]])
e, x = powerMethod(mat)
print(e)
print(x)

6.701562118716424
[ 1.         -0.74031242]


In [118]:
mat = np.array([[1, 2],
                [3, 4]])
e, x = powerMethod(mat)
print(e)
print(x)

5.372281323269014
[0.45742711 1.        ]


In [119]:
mat = np.array([[2, 3],
                [5, 4]])
e, x = powerMethod(mat)
print(e)
print(x)

7.0
[0.6 1. ]


In [120]:
mat = np.array([[4, 2],
                [1, 3]])
e, x = powerMethod(mat)
print(e)
print(x)

5.0
[1.  0.5]
